# Chain of Thought Prompting

[Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)这篇文章提出了一种 CoT prompting 的方法，通过在 prompt 中添加推理步骤，来提高 Large Language Model 的推理能力。

接下来我们将对比一下使用 CoT prompting 和不使用 CoT prompting 的效果，直观感受其效果。

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

os.environ["DEEPSEEK_API_KEY"] = "your_api_key"

# 初始化语言模型
llm = ChatOpenAI(
    model="deepseek-chat",
    base_url="https://api.deepseek.com",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)


In [ ]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="简洁地回答下面的问题: {question}."
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="逐步简洁地回答下面的问题: {question}"
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "我去了市场买了10个苹果。我给了邻居2个苹果，给了修理工2个苹果。然后我又去买了5个苹果，并吃了一个。我还剩下多少个苹果？"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

可以看到，在这种简单的问题上，这样一个 Zero-Shot 的 CoT prompting 已经展现了不一样的效果，当然，我们在这里为了展示 CoT 效果做了一点取巧，那就是添加了所谓的“简洁地”这样的定语，其实我们采用的 DeepSeek 模型过于强大，在这种简单的推理问题上，已经无需显式说明要 step by step 地推理。

你可以尝试一下仅使用“回答下面的问题：”这样的方式，可能得出来的结果详细的超出你的想象！

In [ ]:
prompt = PromptTemplate(
    input_variables=["question"],
    template="回答下面的问题: {question}."
)

chain = prompt | llm

question = "我去了市场买了10个苹果。我给了邻居2个苹果，给了修理工2个苹果。然后我又去买了5个苹果，并吃了一个。我还剩下多少个苹果？"

response = chain.invoke(question).content

print(response)

当然，你也可以自己规定自己想要的求解模板，来激发模型的 CoT 能力，按照你的要求去一步步求解问题的答案。

In [ ]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""根据下面的步骤逐步求解问题的答案:
1. 说明你要计算的内容
2. 如果采用了公式，请列出你要用到的公式
3. 根据问题和公式进行建模计算
4. 解释最后的结果

问题: {question}

答案:"""
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "一间学表演荡秋千，己知秋千的两根绳长均为10m，该同学和秋千踏板的总质最约为5Okg。绳的质量忽略不计。当该同学荡到秋千支架的正下方时，速度大小为8m/s，此时每根绳子平均承受的拉力约为多少"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)